<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/XGBoost%2BBayesain_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/twitter_disaster_tweets/master/train.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train.shape

(7613, 5)

DATA CLEANING

In [5]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [0]:
#Drop keyword and location columns

In [0]:
train = train.drop(['id'], axis=1)

In [0]:
train = train.drop(['keyword'], axis=1)

In [0]:
train = train.drop(['location'], axis=1)

In [10]:
train.isnull().sum()

text      0
target    0
dtype: int64

In [11]:
train.shape

(7613, 2)

In [0]:
test = pd.read_csv('https://raw.githubusercontent.com/bs3537/twitter_disaster_tweets/master/test.csv')
test = test.drop(['id'], axis=1)
test = test.drop(['keyword'], axis=1)
test = test.drop(['location'], axis=1)

In [0]:
#Data cleaning

import re

def text_process(text):
    '''Make text lowercase, remove text in square brackets,remove all single character, Substituting multiple spaces with single space,removing all special characters,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]#', '', text)
    #text = re.sub(r'\W', ' ', str(X[text]))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    #text = re.sub('[%s]' % re.escape(str.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('©', '', text)
    text = re.sub('@', '', text)
    text = re.sub('#', '', text)
    text = re.sub('ûò', '', text)
    text = re.sub('!', '', text)
    text = re.sub('&', '', text)
 
    #text = re.sub('?', '', text)
    return text

In [0]:
# Applying the cleaning function to both test and training datasets
train['text'] = train['text'].apply(lambda x: text_process(x))
test['text'] = test['text'].apply(lambda x: text_process(x))

In [15]:
train.head()

,text,target
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la ronge sask. canada,1
2,all residents asked to 'shelter in place' are ...,1
3,", people receive wildfires evacuation orders i...",1
4,just got sent this photo from ruby alaska as s...,1


In [16]:
test.head()

,text
0,just happened terrible car crash
1,"heard about earthquake is different cities, st..."
2,"there is forest fire at spot pond, geese are f..."
3,apocalypse lighting. spokane wildfires
4,typhoon soudelor kills in china and taiwan


INSTALL SPACY LIBRARIES

In [0]:
pip install spacy

In [0]:
import spacy

In [0]:
#install spacy english pretrained model

!python -m spacy download en_core_web_lg

In [0]:
nlp = spacy.load("en_core_web_lg")

In [0]:
doc = nlp("Twitter disaster tweets dataset")

In [22]:
nlp_vector = doc.vector
print(len(nlp_vector))

300


In [0]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [0]:
#Vectorize predictor columns using spacy

VECTORIZE USING SPACY

In [0]:
#Define X and y train vectors and vectorize using spacy

In [0]:
X_train = train['text']
X_train_spacy = get_word_vectors(X_train)

In [0]:
y_train = train['target']

In [0]:
X_test = test['text']
X_test_spacy = get_word_vectors(X_test)

XGBOOST CLASSIFER MODEL

In [30]:
pip install bayesian-optimization

In [31]:
!git clone https://github.com/rmcantin/bayesopt

Cloning into 'bayesopt'...
remote: Enumerating objects: 2186, done.
remote: Counting objects: 100% (2186/2186), done.
remote: Compressing objects: 100% (894/894), done.
remote: Total 7843 (delta 936), reused 2171 (delta 929), pack-reused 5657
Receiving objects: 100% (7843/7843), 16.07 MiB | 32.46 MiB/s, done.
Resolving deltas: 100% (3540/3540), done.


In [0]:
import xgboost as xgb
from xgboost import XGBClassifier

In [0]:
#Importing necessary libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import classification_report

In [0]:

#Converting the dataframe into XGBoost’s Dmatrix object
dtrain = xgb.DMatrix(X_train_spacy, label=y_train)

In [0]:
#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, n_estimators, learning_rate):
  params = {'max_depth': int(max_depth), 'gamma': gamma,  'n_estimators': int(n_estimators),
            'objective': 'binary:logistic',
            'learning_rate':learning_rate,
            'subsample': 0.8,
            'eta': 0.1, 'eval_metric': 'auc'}

    #Cross validating with the specified parameters in 5 folds and 100 iterations
  cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=5)
    #Return the test AUC on CV
  return cv_result['test-auc-mean'].iloc[-1]

In [50]:
#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (1, 15),
                                             'gamma': (0, 1),
                                             'learning_rate':(0,1),
                                             'n_estimators':(100,1000)
                                            })

#performing Bayesian optimization for 10 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=10, init_points=8, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.8568   |  0.9325   |  0.3296   |  8.574    |  653.5    |
|  2        |  0.8304   |  0.7788   |  0.7917   |  3.081    |  966.9    |
|  3        |  0.8428   |  0.7682   |  0.07817  |  1.241    |  986.2    |
|  4        |  0.8593   |  0.6789   |  0.2807   |  14.85    |  695.6    |
|  5        |  0.8587   |  0.8148   |  0.38     |  2.671    |  515.2    |
|  6        |  0.8622   |  0.4596   |  0.2916   |  11.65    |  782.9    |
|  7        |  0.8639   |  0.8253   |  0.1714   |  14.68    |  530.5    |
|  8        |  0.8297   |  0.03472  |  0.9359   |  6.874    |  239.9    |
|  9        |  0.8635   |  0.4313   |  0.1283   |  14.15    |  100.8    |
|  10       |  0.8672   |  0.8568   |  0.09073  |  14.77    |  423.7    |
|  11       |  0.8457   |  0.9823   |  0.09133  |  1.27     |  753.2    |
|  12       |  0.8471   |  0.5568   | 

In [51]:
#Extracting the best parameters
params = xgb_bo.max['params']
print(params)

{'gamma': 0.8567988386812423, 'learning_rate': 0.09072846971154513, 'max_depth': 14.76798072911884, 'n_estimators': 423.6681475969234}


In [0]:
#Best hyperparamters: {'gamma': 0.8567988386812423, 'learning_rate': 0.09072846971154513, 'max_depth': 14.76798072911884, 'n_estimators': 423.6681475969234}

In [0]:

#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])


In [0]:
#Initialize an XGBClassifier with the tuned parameters and fit the training data
from xgboost import XGBClassifier
classifier2 = XGBClassifier(**params).fit(np.array(X_train_spacy), y_train)

Predict on test dataset

In [0]:
#predicting for training set
preds = classifier2.predict(X_test_spacy)

In [58]:
preds

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
#Making submission
test = pd.read_csv('https://raw.githubusercontent.com/bs3537/twitter_disaster_tweets/master/test.csv')
submission = pd.DataFrame({'id': test['id'], 'target':preds})
#submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [61]:
submission.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [0]:
submission.to_csv('disaster_submission_xgboost_bayesian_best_parameters.csv', index=False)